# Préparation des données pour Projet Final JO - Inégalités H/F

Pour ce projet, je m'occupe de la partie sur l'étude des inégalités Hommes/Femmes aux Jeux Olympiques.  
J'utilise les 4 tables suivantes :
- Les tables "athletes_events.csv" et "noc_regions.csv" du jeu de données "120 years of Olympic history", disponible sur Kaggle :  
    https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results?resource=download
- La table "gender-inequality-index-from-the-human-development-report.csv", disponible sur Our World in Data :  
    https://ourworldindata.org/grapher/gender-inequality-index-from-the-human-development-report
- La table "countrycodes.csv", disponible sur WorldData.info :  
    https://www.worlddata.info/countrycodes.php

### Importation et nettoyage des fichiers

1. Fichier Athlètes JO

In [1]:
# Importation de la bibliothèque Pandas
import pandas as pd

# Importer les données dans un dataframe
olympic_games = pd.read_csv("./athlete_events.csv", sep = ",")

# Ne garder que les jeux d'été
olympic_games = olympic_games[olympic_games['Season'] == 'Summer']

# Ne garder que les colonnes utiles pour l'étude
olympic_games = olympic_games[['Sex', 'Team', 'NOC', 'Year', 'Sport', 'Event', 'Medal']]

# Création d'ID (concaténation du NOC et année) pour lier à la table des GII
olympic_games['ID'] = olympic_games['NOC'] + olympic_games['Year'].astype(str)

olympic_games.head()

,Sex,Team,NOC,Year,Sport,Event,Medal,ID
0,M,China,CHN,1992,Basketball,Basketball Men's Basketball,NaN,CHN1992
1,M,China,CHN,2012,Judo,Judo Men's Extra-Lightweight,NaN,CHN2012
2,M,Denmark,DEN,1920,Football,Football Men's Football,NaN,DEN1920
3,M,Denmark/Sweden,DEN,1900,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,DEN1900
26,F,Netherlands,NED,1932,Athletics,Athletics Women's 100 metres,NaN,NED1932


2. Fichier NOC/Régions

In [2]:
# Importer les données dans un dataframe
noc_regions = pd.read_csv("./noc_regions.csv", sep = ",")

# Trouver les valeurs manquantes dans les colonnes NOC et Region
missing_noc_region = noc_regions[noc_regions[['NOC', 'Region']].isnull().any(axis=1)]
print(missing_noc_region)

# Remplacer les valeurs manquantes dans Region par les valeurs de Notes
noc_regions['Region'].fillna(noc_regions['Notes'], inplace=True)

# Supprimer la colonne Notes
noc_regions = noc_regions.drop('Notes', axis=1)

     NOC Region                 Notes
168  ROT    NaN  Refugee Olympic Team
208  TUV    NaN                Tuvalu
213  UNK    NaN               Unknown


In [3]:
# En visualisation rapide sur Tableau (Map), on voit que la Bolivie n'apparait pas
bolivia_region = noc_regions[noc_regions['NOC'] == 'BOL']
if bolivia_region.empty:
    print("No region information available for NOC code 'BOL'")
else:
    print(bolivia_region)

# Il y a une erreur sur le nom "Boliva" au lieu de "Bolivia"
if not bolivia_region.empty:
    noc_regions.loc[noc_regions['NOC'] == 'BOL', 'Region'] = 'Bolivia'

    NOC  Region
27  BOL  Boliva


In [4]:
# On voit sur Data Wrangler qu'il y a plus de codes NOC que de regions, c'est normal
multiple_noc_regions = noc_regions.groupby('Region')['NOC'].unique().reset_index()
multiple_noc_regions = multiple_noc_regions[multiple_noc_regions['NOC'].str.len() > 1]

print(multiple_noc_regions)

             Region                   NOC
10        Australia            [ANZ, AUS]
33           Canada            [CAN, NFL]
39            China            [CHN, HKG]
48   Czech Republic       [BOH, CZE, TCH]
67          Germany  [FRG, GDR, GER, SAA]
69           Greece            [CRT, GRE]
110        Malaysia       [MAL, MAS, NBO]
152          Russia       [EUN, RUS, URS]
162          Serbia       [SCG, SRB, YUG]
180           Syria            [SYR, UAR]
188        Trinidad            [TTO, WIF]
203         Vietnam            [VIE, VNM]
206           Yemen       [YAR, YEM, YMD]
208        Zimbabwe            [RHO, ZIM]


In [5]:
# On enregistre le dataframe nettoyé à utiliser dans Tableau
noc_regions.to_csv('final_data/noc_regions_clean.csv', index=False)

noc_regions.head()

,NOC,Region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


3. Fichier Codes pays

In [6]:
# Importer les données dans un dataframe
noc_codes = pd.read_csv("./countrycodes.csv", sep=",", encoding="ISO-8859-1")

noc_codes.head()

,Country,ISO 3166-1 alpha2,ISO 3166-1 alpha3,ISO 3166-1 numeric,IOC,Fips 10,License Plate,Domain
0,Afghanistan,AF,AFG,4,AFG,AF,AFG,.af
1,Åland Islands,AX,ALA,248,NaN,NaN,AX,.ax
2,Albania,AL,ALB,8,ALB,AL,AL,.al
3,Algeria,DZ,DZA,12,ALG,AG,DZ,.dz
4,American Samoa,AS,ASM,16,ASA,AQ,USA,.as


In [7]:
# Ne garder que les deux colonnes Code et NOC puis les renommer
noc_codes = noc_codes[['ISO 3166-1 alpha3', 'IOC']]
noc_codes = noc_codes.rename(columns={'ISO 3166-1 alpha3': 'Code', 'IOC': 'NOC'})

# Supprimer les valeurs manquantes (45 sur Data Wrangler)
noc_codes = noc_codes.dropna()

# Enregistrer le nouveau fichier
noc_codes.to_csv("final_data/noc_codes_clean.csv", index=False)

noc_codes.head()

,Code,NOC
0,AFG,AFG
2,ALB,ALB
3,DZA,ALG
4,ASM,ASA
5,AND,AND


4. Fichier Gender Inequality Index (GII)

In [8]:
# Importer les données dans un dataframe
gii_country = pd.read_csv("./gender-inequality-index-from-the-human-development-report.csv", sep = ",")

# Enlever les lignes sans Code
gii_country.dropna(subset=['Code'], inplace=True)

# Renommer la colonne des index
gii_country.rename(columns={'Gender Inequality Index': 'GII'}, inplace=True)

# Ajouter une colonne avec les codes NOC correspondants à Code pour chaque ligne (fusionner avec noc_codes)
gii_country = gii_country.merge(noc_codes, left_on='Code', right_on='Code', how='left')

# Remplacer la valeur manquante pour South Sudan par SSD (vérifié sur site Olympics.com et table noc_regions)
gii_country.loc[gii_country['Entity'] == 'South Sudan', 'NOC'] = 'SSD'

gii_country.head()

,Entity,Code,Year,GII,NOC
0,Afghanistan,AFG,2005,0.748,AFG
1,Afghanistan,AFG,2006,0.749,AFG
2,Afghanistan,AFG,2007,0.752,AFG
3,Afghanistan,AFG,2008,0.755,AFG
4,Afghanistan,AFG,2009,0.755,AFG


In [9]:
# Vérifier et supprimer les dernières valeurs manquantes (concernent l'entité World)
missing_values = gii_country[gii_country.isnull().any(axis=1)]
print(missing_values)
gii_country.dropna(inplace=True)

     Entity      Code  Year    GII  NOC
4801  World  OWID_WRL  1990  0.581  NaN
4802  World  OWID_WRL  1991  0.581  NaN
4803  World  OWID_WRL  1992  0.580  NaN
4804  World  OWID_WRL  1993  0.580  NaN
4805  World  OWID_WRL  1994  0.578  NaN
4806  World  OWID_WRL  1995  0.577  NaN
4807  World  OWID_WRL  1996  0.574  NaN
4808  World  OWID_WRL  1997  0.571  NaN
4809  World  OWID_WRL  1998  0.565  NaN
4810  World  OWID_WRL  1999  0.561  NaN
4811  World  OWID_WRL  2000  0.557  NaN
4812  World  OWID_WRL  2001  0.553  NaN
4813  World  OWID_WRL  2002  0.548  NaN
4814  World  OWID_WRL  2003  0.543  NaN
4815  World  OWID_WRL  2004  0.535  NaN
4816  World  OWID_WRL  2005  0.526  NaN
4817  World  OWID_WRL  2006  0.520  NaN
4818  World  OWID_WRL  2007  0.514  NaN
4819  World  OWID_WRL  2008  0.511  NaN
4820  World  OWID_WRL  2009  0.506  NaN
4821  World  OWID_WRL  2010  0.504  NaN
4822  World  OWID_WRL  2011  0.500  NaN
4823  World  OWID_WRL  2012  0.495  NaN
4824  World  OWID_WRL  2013  0.490  NaN


In [10]:
# Compter les valeurs NOC dans les deux dataframes
gii_country_count = gii_country['NOC'].nunique()
print("Number of unique NOC in gii_country:", gii_country_count)
olympic_games_count = olympic_games['NOC'].nunique()
print("Number of unique NOC in olympic_games:", olympic_games_count)

Number of unique NOC in gii_country: 170
Number of unique NOC in olympic_games: 230


In [11]:
# Trouver quelles valeurs de la table GII ne sont pas présents dans JO
missing_noc = gii_country[~gii_country['NOC'].isin(olympic_games['NOC'])]
print("NOC values from gii_country not present in olympic_games:")
print(missing_noc)

NOC values from gii_country not present in olympic_games:
       Entity Code  Year    GII  NOC
2419  Lebanon  LBN  2007  0.422  LBN
2420  Lebanon  LBN  2008  0.420  LBN
2421  Lebanon  LBN  2009  0.444  LBN
2422  Lebanon  LBN  2010  0.445  LBN
2423  Lebanon  LBN  2011  0.452  LBN
2424  Lebanon  LBN  2012  0.453  LBN
2425  Lebanon  LBN  2013  0.458  LBN
2426  Lebanon  LBN  2014  0.462  LBN
2427  Lebanon  LBN  2015  0.464  LBN
2428  Lebanon  LBN  2016  0.464  LBN
2429  Lebanon  LBN  2017  0.462  LBN
2430  Lebanon  LBN  2018  0.434  LBN
2431  Lebanon  LBN  2019  0.432  LBN
2432  Lebanon  LBN  2020  0.433  LBN
2433  Lebanon  LBN  2021  0.432  LBN


In [12]:
# Voir ce qui est enregistré pour les lignes de l'équipe Lebanon
lebanon_data = olympic_games[olympic_games['Team'] == 'Lebanon']
if lebanon_data.empty:
    print("No data available for Team 'Lebanon' in olympic_games.")
else:
    print(lebanon_data)

       Sex     Team  NOC  Year      Sport  \
240      M  Lebanon  LIB  1988  Athletics   
241      M  Lebanon  LIB  1988  Athletics   
821      M  Lebanon  LIB  1968    Cycling   
822      M  Lebanon  LIB  1968    Cycling   
823      M  Lebanon  LIB  1968    Cycling   
...     ..      ...  ...   ...        ...   
266369   M  Lebanon  LIB  1988    Fencing   
266370   M  Lebanon  LIB  1992    Fencing   
266371   M  Lebanon  LIB  1992    Fencing   
267137   M  Lebanon  LIB  1988    Cycling   
267138   M  Lebanon  LIB  1992    Cycling   

                                        Event Medal       ID  
240                Athletics Men's 400 metres   NaN  LIB1988  
241                Athletics Men's 800 metres   NaN  LIB1988  
821       Cycling Men's Road Race, Individual   NaN  LIB1968  
822                      Cycling Men's Sprint   NaN  LIB1968  
823     Cycling Men's 1,000 metres Time Trial   NaN  LIB1968  
...                                       ...   ...      ...  
266369         Fen

In [13]:
# Remplacer le NOC de Lebanon par LIB car plus simple (pas d'autres entités avec LIB dans le dataframe)
gii_country.loc[gii_country['Entity'] == 'Lebanon', 'NOC'] = 'LIB'

# Création de l'ID (concaténation du NOC et année) pour lier avec la table JO
gii_country['ID'] = gii_country['NOC'] + gii_country['Year'].astype(str)

# Enregistrer le dataframe dans un fichier csv
gii_country.to_csv('final_data/gii_country_clean.csv', index=False)

gii_country.head()

,Entity,Code,Year,GII,NOC,ID
0,Afghanistan,AFG,2005,0.748,AFG,AFG2005
1,Afghanistan,AFG,2006,0.749,AFG,AFG2006
2,Afghanistan,AFG,2007,0.752,AFG,AFG2007
3,Afghanistan,AFG,2008,0.755,AFG,AFG2008
4,Afghanistan,AFG,2009,0.755,AFG,AFG2009


In [14]:
# Voir combien d'ID vont pouvoir être comparés pour les GII et les JO
matching_ids = gii_country['ID'].isin(olympic_games['ID'])
num_matching_ids = matching_ids.sum()
num_non_matching_ids = len(gii_country) - num_matching_ids

print("Number of matching IDs:", num_matching_ids)
print("Number of non-matching IDs:", num_non_matching_ids)

Number of matching IDs: 1041
Number of non-matching IDs: 3848


### Préparation des données et calculs

Ajout d'une colonne "Type" pour différentier les sports collectifs/individuels

In [15]:
# Créer une liste des events collectifs basés sur les médailles d'or
gold_events = olympic_games.loc[olympic_games['Medal'] == "Gold"]
# Lorsque plusieurs athlètes reçoivent des médailles d'or sur le même event/équipe/année/sexe
gold_duplicates = gold_events.duplicated(subset=["Team", "Sex", "Event", "Medal", "Year"], keep = False)
team_events = gold_events[gold_duplicates]
team_events = team_events["Event"].unique()

# Créer une colonne "Type" avec valeurs par défaut "Individual" 
olympic_games['Type'] = 'Individual'

# Change le type à "Team" pour les sports inclus dans l'array
olympic_games.loc[olympic_games['Event'].isin(team_events), 'Type'] = 'Team'

olympic_games.head()

,Sex,Team,NOC,Year,Sport,Event,Medal,ID,Type
0,M,China,CHN,1992,Basketball,Basketball Men's Basketball,NaN,CHN1992,Team
1,M,China,CHN,2012,Judo,Judo Men's Extra-Lightweight,NaN,CHN2012,Individual
2,M,Denmark,DEN,1920,Football,Football Men's Football,NaN,DEN1920,Team
3,M,Denmark/Sweden,DEN,1900,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,DEN1900,Team
26,F,Netherlands,NED,1932,Athletics,Athletics Women's 100 metres,NaN,NED1932,Individual


Grouper par ID, Event, Sex et calculer le nombre d'athlètes (Total, Femmes, Hommes)

In [16]:
# Group by Sex, ID, and Event and calculate the desired metrics
grouped_df = olympic_games.groupby(['NOC', 'Year', 'Sex', 'Event']).agg(
    ID = ('ID', 'first'),
    Type = ('Type', 'first'),
    Total_Athletes = ('ID', 'size'),
    Female_Athletes = ('Sex', lambda x: (x == 'F').sum()),
    Male_Athletes = ('Sex', lambda x: (x == 'M').sum())
).reset_index()

# Convert the desired columns to integers
grouped_df['Total_Athletes'] = grouped_df['Total_Athletes'].astype(int)
grouped_df['Female_Athletes'] = grouped_df['Female_Athletes'].astype(int)
grouped_df['Male_Athletes'] = grouped_df['Male_Athletes'].astype(int)

# Enregistrer le nouveau dataframe dans un fichier csv
grouped_df.to_csv('final_data/event_genders.csv', index=False)

# Display the resulting dataframe
grouped_df.head()

,NOC,Year,Sex,Event,ID,Type,Total_Athletes,Female_Athletes,Male_Athletes
0,AFG,1936,M,Athletics Men's 100 metres,AFG1936,Individual,1,0,1
1,AFG,1936,M,Athletics Men's Long Jump,AFG1936,Individual,1,0,1
2,AFG,1936,M,Athletics Men's Shot Put,AFG1936,Individual,1,0,1
3,AFG,1936,M,Hockey Men's Hockey,AFG1936,Team,13,0,13
4,AFG,1948,M,Football Men's Football,AFG1948,Team,11,0,11


Ajout d'une colonne nombre de femmes dans la table GII

In [17]:
# Group by ID and count occurrences of each ID
grouped_data = olympic_games.groupby('ID').size().reset_index(name='Total')

# Calculate the count of women for each ID
women_count = olympic_games[olympic_games['Sex'] == 'F'].groupby('ID').size().reset_index(name='Female_Count')

# Calculate the percentage of women for each ID
grouped_data = grouped_data.merge(women_count, on='ID', how='left')
grouped_data['Female_Ratio'] = (grouped_data['Female_Count'] / grouped_data['Total'])

# Merge with gii_country dataframe based on ID
gii_country = gii_country.merge(grouped_data[['ID', 'Female_Ratio']], on='ID', how='left')

# Remove rows with missing data
gii_country.dropna(inplace=True)

gii_country.to_csv('final_data/gii_country_final.csv', index=False)
gii_country.head()

,Entity,Code,Year,GII,NOC,ID,Female_Ratio
3,Afghanistan,AFG,2008,0.755,AFG,AFG2008,0.250000
7,Afghanistan,AFG,2012,0.738,AFG,AFG2012,0.166667
11,Afghanistan,AFG,2016,0.692,AFG,AFG2016,0.333333
18,Albania,ALB,2000,0.319,ALB,ALB2000,0.600000
22,Albania,ALB,2004,0.301,ALB,ALB2004,0.285714
